## DFO get sampling platform summary

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle

from sqlalchemy import create_engine, case
from sqlalchemy.orm import create_session 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.sql import and_, or_, not_, func

mpl.rc('xtick', labelsize=10)
mpl.rc('ytick', labelsize=10)
mpl.rc('legend', fontsize=10)
mpl.rc('axes', titlesize=10)
mpl.rc('axes', labelsize=10)
mpl.rc('figure', titlesize=10)
mpl.rc('font', size=10)
%matplotlib inline

In [7]:
basedir='/ocean/eolson/MEOPAR/obs/DFOOPDB/' 
dbname='DFO_OcProfDB.sqlite'
datelims=(dt.datetime(2015,1,1),dt.datetime(2018,1,1))

engine = create_engine('sqlite:///' + basedir + dbname, echo = False)
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
# existing tables:
StationTBL=Base.classes.StationTBL
ObsTBL=Base.classes.ObsTBL
CalcsTBL=Base.classes.CalcsTBL
JDFLocsTBL=Base.classes.JDFLocsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)
start_y=datelims[0].year
start_m=datelims[0].month
start_d=datelims[0].day
end_y=datelims[1].year
end_m=datelims[1].month
end_d=datelims[1].day
qry=session.query(StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
              StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),
              StationTBL.Lat,StationTBL.Lon,StationTBL.PLATFORM,
             ObsTBL.Pressure,ObsTBL.Depth,ObsTBL.Ammonium,ObsTBL.Ammonium_units,ObsTBL.Chlorophyll_Extracted,
             ObsTBL.Chlorophyll_Extracted_units,ObsTBL.Nitrate_plus_Nitrite.label('N'),
              ObsTBL.Silicate.label('Si'),ObsTBL.Silicate_units).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsID==ObsTBL.ID).filter(and_(or_(StationTBL.StartYear>start_y,
                                                                 and_(StationTBL.StartYear==start_y, StationTBL.StartMonth>start_m),
                                                                 and_(StationTBL.StartYear==start_y, StationTBL.StartMonth==start_m, StationTBL.StartDay>=start_d)),
                                                             or_(StationTBL.StartYear<end_y,
                                                                 and_(StationTBL.StartYear==start_y,StationTBL.StartMonth<start_m),
                                                                 and_(StationTBL.StartYear==start_y,StationTBL.StartMonth==start_m, StationTBL.StartDay<=start_d)),
                                                            StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                                            or_(ObsTBL.Silicate>=0,
                                                                ObsTBL.Nitrate_plus_Nitrite>=0)))
qsub=qry.subquery()

In [9]:
q2=session.query(qsub.c.PLATFORM,func.count(qsub.c.PLATFORM).label('Count')).group_by(qsub.c.PLATFORM)
df2=q2.all()

In [10]:
df2

[('John P. Tully', 670),
 ('Sea Crest', 6),
 ('Vector', 3220),
 ('W.E. Ricker', 64)]